### Evaluation step1, date range affectiveness, date range = 1 day

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 0  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 2 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 1  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath'+ str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 3 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 2  # change savePath and date_range to create different groundtruth


loadPath = 'yourloadpath'+ str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 4 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 3  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath'+ str(date_range+1) + '/'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 5 days (weekly)

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 4  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 6 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 5  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath' + str(date_range+1) + '/'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 7 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 6  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath' + str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 8 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 7  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath' + str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 9 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 8  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath' + str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

### Evaluation step1, date range affectiveness, date range = 10 days

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

processedList = []
errorList = []
# outfile = TemporaryFile()

date_range = 9  # change savePath and date_range to create different groundtruth

loadPath = 'yourloadpath' + str(date_range+1) + '/'
savePath = 'yoursavepath'


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = random.sample(fileList, 100)  # random select 
# fileList = ['002645.SZ.csv', '002114.SZ.csv', '603032.SH.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
a_s = np.zeros([1, 56]) # calculated according to X shape
a_r = np.zeros([1, 56]) # calculated according to X shape
a_gs = np.zeros([1, 56]) # calculated according to X shape

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    data = data.replace([np.inf, -np.inf], 0)
    
#     #debug
#     fileName = savePath + f
#     data.to_csv(fileName, encoding = 'utf-8', index = False)
    
    y = data['w_up_down'].values.tolist()
    X = data.iloc[:,2:58].copy() # To avoid the case where changing X also changes data, only excluded the ts code and trading date
    X.fillna(0, inplace=True)
    X = X.values
    
    y0 = np.zeros([1,1])
    y = np.append(y, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
    y = np.delete(y, (0), axis=0) # delete first row to ensure the lenth is correct
    
    X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
    y = y[:-1]  # delete the last row of y because the last row of label is y0
    
    try:
        X = scaler.fit_transform(X)

        estimator = SVR(kernel="linear")
        selector = RFECV(estimator, step=1, cv=5)
        selector = selector.fit(X, y)
        
        condition = selector.support_

        if len(condition)%56 == 0:
            a_s = np.vstack((a_s, selector.support_))
            a_r = np.vstack((a_r, selector.ranking_))
            
            a_gs = np.vstack((a_gs, selector.grid_scores_))

            
#             np.save(savePath+'a_s_weekly.npy', a_s)
#             np.save(savePath+'a_r_weekly.npy', a_r)
            
            processedList.append(f)
        else:
            errorList.append(f)
    except:
        errorList.append(f)

# feature ranking array

a_s = np.delete(a_s, (0), axis=0) # delete first row
a_r = np.delete(a_r, (0), axis=0) # delete first row
a_gs = np.delete(a_gs, (0), axis=0) # delete first row

In [ ]:
data.columns[2:58]

In [ ]:
idxnameList = data.columns[2:58].tolist()
idxsptList = np.sum(a_s, axis=0).tolist()
idxrkList = np.sum(a_r, axis=0).tolist()
idxgsList = np.mean(a_gs, axis=0).tolist()
a_gs_mean = np.mean(a_gs, axis=0)

In [ ]:
df_RFE = pd.DataFrame(
    {'Name': idxnameList,
     'Support': idxsptList,
     'Rank': idxrkList,
     'Grid_Score': idxgsList
    })

In [ ]:
df_RFE.sort_values('Rank')
temp = df_RFE.sort_values('Rank')['Name'].tolist()
temp[0:15]

In [ ]:
df_RFE.sort_values('Support')
temp = df_RFE.sort_values('Support', ascending = False)['Name'].tolist()
temp[0:15]

In [ ]:
# plot
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(a_gs_mean) + 1), a_gs_mean)
plt.show()

In [ ]:
df_RFE

In [ ]:
fileName = savePath + 'RFE_' + str(date_range + 1) + 'days_prediction.csv'
df_RFE.to_csv(fileName, encoding = 'utf-8', index = False)

# Test section from now on

In [ ]:
# Copy paste test section
# minmaxscaler and fillna
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

new_df = pd.DataFrame()


fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
# fileList = ['RFE_7days_prediction.csv']

for f in fileList:
    fileName = loadPath + f
    try:
        data = pd.read_csv(fileName)
        col = f[4:9]
        new_df[col] = data['Grid_Score']
    except:
        print(f)
        
fileName = savePath + 'RFE_curve.csv'
new_df.to_csv(fileName, encoding = 'utf-8', index = False)

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='username', api_key='apikey')

# Create random data with numpy
import numpy as np

feature_num = new_df.index.values.tolist()


trace0 = go.Scatter(
    x = feature_num,
    y = new_df['1days'],
    name = '1 day',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace1 = go.Scatter(
    x = feature_num,
    y = new_df['2days'],
    name = '2 days',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace2 = go.Scatter(
    x = feature_num,
    y = new_df['3days'],
    name = '3 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace3 = go.Scatter(
    x = feature_num,
    y = new_df['4days'],
    name = '4 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)


trace4 = go.Scatter(
    x = feature_num,
    y = new_df['5days'],
    name = '5 days',
    line = dict(
        color = ('rgb(255, 191, 0)'),
        width = 2)
)

trace5 = go.Scatter(
    x = feature_num,
    y = new_df['6days'],
    name = '6 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace6 = go.Scatter(
    x = feature_num,
    y = new_df['7days'],
    name = '7 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace7 = go.Scatter(
    x = feature_num,
    y = new_df['8days'],
    name = '8 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace8 = go.Scatter(
    x = feature_num,
    y = new_df['9days'],
    name = '9 days',
    line = dict(
        color = ('rgb(220, 220, 220)'),
        width = 2,
        dash = 'dash' # dash options include 'dash', 'dot', and 'dashdot'
    )
)

trace9 = go.Scatter(
    x = feature_num,
    y = new_df['10day'],
    name = '10 days',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2
    )
)

data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]

layout = dict(title = 'How Term Lengths Affect RFE',
              xaxis = dict(title = 'Number of features selected'),
              yaxis = dict(title = 'Cross validation score (nb of correct classifications)'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='RFE Evaluation')